In [706]:
import re
import tkinter as tk

respond = []
sub = []
#instructions
operation = ['add','sub','lw','sw','addi','addu','addiu','sll','srl','and','or','nor',
             'beq','bne','slt','slti','sltu','j','jal','jr']
#zreo,for IType const,arguments,temp,contents for later,temp,OS,...
reg = {'$zero': 0,
        '$at':None,
        '$v0':None,'$v1':None,
        '$a0':None,'$a1':None,'$a2':None,'$a3':None,
        '$t0':None,'$t1':None,'$t2':None,'$t3':None,'$t4':None,'$t5':None,'$t6':None,'$t7':None,
        '$s0':None,'$s1':None,'$s2':None,'$s3':None,'$s4':None,'$s5':None,'$s6':None,'$s7':None,
        '$t8':None,'$t9':None,
        '$k0':None,'$k1':None,
        '$gp':None,'$sp':None,'$fp':None,'$ra':None }
reg_table = {'$0':'$zero',
            '$1':'$at',
            '$2':'$v0','$3':'$v1',
            '$4':'$a0','$5':'$a1','$6':'$a2','$7':'$a3',
            '$8':'$t0','$9':'$t1','$10':'$t2','$11':'$t3','$12':'$t4','$13':'$t5','$14':'$t6','$15':'$t7',
            '$16':'$s0','$17':'$s1','$18':'$s2','$19':'$s3','$20':'$s4','$21':'$s5','$22':'$s6','$23':'$s7',
            '$24':'$t8','$25':'$t9',
            '$26':'$k0','$27':'$k1',
            '$28':'$gp','$29':'$sp','$30':'$fp','$31':'$ra'}



In [707]:
def isNum(c):
    try:    # 如果能运⾏ float(s) 语句，返回 True（字符串 s 是浮点数）        
        float(c)        
        return True    
    except ValueError:  # ValueError 为 Python 的⼀种标准异常，表⽰"传⼊⽆效的参数"        
        pass  # 如果引发了 ValueError 这种异常，不做任何事情（pass：不做任何事情，⼀般⽤做占位语句）    
    try:        
        import unicodedata  # 处理 ASCII 码的包        
        unicodedata.numeric(c)  # 把⼀个表⽰数字的字符串转换为浮点数返回的函数        
        return True    
    except (TypeError, ValueError):        
        pass    
        return False

In [708]:
def parse(User_input):           #split and classify line to op or reg
    tokens=[]
    splitedLine = re.split('[#:(,\s]+',User_input)
    for c in splitedLine:
        if c != "":
            if c.lower() in operation:
                tokens.append([c,"OPRT"])
            elif isNum(c):
                c = int(c)
                tokens.append([c,"NUM"])
            elif c in reg_table or c in reg_table.values():
                tokens.append([c,"REG"])
            else:
                tokens.append([c,"Unknown"])
        else:
            return False
    return tokens

In [709]:
def executeInstr(tokens):
    global respond
    respond = []
    if tokens==False:return False

    if(tokens[0][1]=="OPRT"):
        if len(tokens) >= 4:
            if(tokens[0][0].lower() == "add"):
                add(tokens)
            elif(tokens[0][0].lower() == "sub"):
                sub(tokens)
            elif(tokens[0][0].lower() == "lw"):
                lw(tokens)
            elif(tokens[0][0].lower() == "sw"):
                sw(tokens)
            elif(tokens[0][0].lower() == "addi"):
                addi(tokens)
            elif(tokens[0][0].lower() == "addu"):
                addu(tokens)
            elif(tokens[0][0].lower() == "addiu"):
                addiu(tokens)
            elif(tokens[0][0].lower() == "sll" or tokens[0][0].lower() == "srl"):
                shift(tokens)
            elif(tokens[0][0].lower() == "and" or tokens[0][0].lower() == "or" or tokens[0][0].lower() == "nor"):
                AndOrNor(tokens)
            elif(tokens[0][0].lower() == "beq" or tokens[0][0].lower() == "bne"):
                branch(tokens)
            elif(tokens[0][0].lower() == "slt"):
                slt(tokens)
            elif(tokens[0][0].lower() == "slti"):
                slti(tokens)
            elif(tokens[0][0].lower() == "sltu"):
                sltu(tokens)
        elif len(tokens) == 2:
            if(tokens[0][0].lower() == "j" or tokens[0][0].lower() == "jal" or tokens[0][0].lower() == "jr"):
                jump(tokens)
        else:
            respond.append(["ERR",f"Error:Expected identifier."])
    else:
        respond.append(["Error",f"Unexpected Operation：{tokens[0][0]}."]) 

In [710]:
def lw(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if (i%2==1)and(tokens[i][1] != "REG"):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        elif(tokens[i][1] == "NUM")and(tokens[i][0] > 65535 or tokens[i][0] < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is out of range."])
            stopExe = True
        elif (tokens[i][1] == "Unknown"):
            respond.append(["Error",f"Unknown identifier {tokens[i][0]} ."])
    respond.append(["operation",tokens[0][0]])
    if stopExe == False:
        respond.append(["source 1",tokens[3][0]])
        address = f"MEM[${tokens[3][0]}+{tokens[2][0]}]"
        respond.append(["memory address",address])

def sw(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):   
        if (i%2==1)and(tokens[i][1] != "REG"):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        elif(tokens[i][1] == "NUM")and(tokens[i][0] > 65535 or tokens[i][0] < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is out of range."])
            stopExe = True
        elif (tokens[i][1] == "Unknown"):
            respond.append(["Error",f"Unknown identifier {tokens[i][0]} ."])
    respond.append(["operation",tokens[0][0]])
    if stopExe == False:
        respond.append(["source 1",tokens[3][0]])
        respond.append(["source 2",tokens[1][0]])
        address = f"MEM[{tokens[3][0]}+{tokens[2][0]}]"
        respond.append(["memory address",address])


In [711]:
def sub(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])
#     Rtype(000000,,,00000,100010)
        
def add(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])
    
def addi(tokens):
    global respond
    stopExe = False
    
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True

    if(tokens[3][1] == "NUM")and(tokens[3][0] > 32767 or tokens[3][0] < -32768):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["immediate value",tokens[3][0]])
            
def addu(tokens):
    global respond
    stopExe = False
    
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    #check unsign
    if stopExe == False:
        if tokens[3][0] in reg_table:
            rt = reg_table[tokens[3][0]]
        else:
            rt = tokens[3][0]
        value = reg[rt]
        if value != None and (value > 65535 or value < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
            stopExe = True 
    
    respond.append(["operation",tokens[0][0]])
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])
            
def addiu(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
            
    if(tokens[3][1] == "NUM")and(tokens[3][0] > 65535 or tokens[3][0] < 0):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["immediate value",tokens[3][0]])

In [712]:
def shift(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
            
    if(tokens[3][1] == "NUM")and (tokens[3][0] > 31 or tokens[i][0] < 0):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["shift amount",tokens[3][0]])

In [713]:
def AndOrNor(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])

In [714]:
def branch(tokens):
    global sub
    global respond
    stopExe = False

    for i in range(1,4,1):
        if i<3 and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if i==3 and tokens[i][1] == "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} can not be a label."])
            stopExe = True
        if i==3 and  tokens[i][1] == "NUM" and tokens[i][0]%4 != 0:
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an address."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
#         sub.append([tokens[3][0],None])
        respond.append(["source 1",tokens[1][0]])
        respond.append(["source 2",tokens[2][0]])
        respond.append(["address",tokens[3][0]])

In [715]:
def slt(tokens):
    global respond
    stopExe = False

    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])
        
def slti(tokens):
    global respond
    stopExe = False
    
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
            
    if(tokens[3][1] == "NUM")and(tokens[3][0] > 32767 or tokens[3][0] < -32768):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["immediate value",tokens[3][0]])
        
def sltu(tokens):
    global respond
    stopExe = False
    
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    #check unsign
    if stopExe == False:
        if tokens[3][0] in reg_table:
            rt = reg_table[tokens[3][0]]
        else:
            rt = tokens[3][0]
        value = reg[rt]
        if value != None and (value > 65535 or value < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
            stopExe = True 
    
    respond.append(["operation",tokens[0][0]])
    if stopExe == False:
        respond.append(["destination",tokens[1][0]])
        respond.append(["source 1",tokens[2][0]])
        respond.append(["source 2",tokens[3][0]])

In [716]:
def jump(tokens):
    global respond
    stopExe = False
        
    if tokens[0][0].lower() != "jr":
        if tokens[1][1] == "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} can not be a label."])
            stopExe = True
        if tokens[1][1] == "NUM" and tokens[1][0]%4 != 0:
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} is not an address."])
            stopExe = True
        if tokens[1][1] == "NUM" and tokens[1][0] > 67108863 :
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} is out of range."])
            stopExe = True 
            
    else:
        if tokens[1][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement,  no suitable register {tokens[1][0]}."])
            stopExe = True
        if tokens[1][0] != "$ra" and tokens[1][0] != "$31":
            respond.append(["Waring",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} is not a return address register."])
    
    respond.append(["operation",tokens[0][0]])
    
    if stopExe == False:
#         sub.append([token[1][0],""])
        respond.append(["address",tokens[1][0]])


In [717]:
def Rtype(op,rs,rt,rd,shamt,funct):
    respond.append(["opcode",op])
    respond.append(["rs",rs])
    respond.append(["rt",rt])
    respond.append(["rd",rd])
    respond.append(["shamt",shamt])
    respond.append(["funct",funct])
def Itype(op,rs,rt,imm,flag):
    respond.append(["opcode",op])
    respond.append(["rs",rs])
    respond.append(["rt",rt])
    if flag==0:
        respond.append(["address",imm])
    else:
        respond.append(["immediate value",imm])
def Jtype(op,address):
    respond.append(["opcode",op])
    respond.append(["address",address])

In [ ]:
# GUI 
User_input=''
window = tk.Tk()

width = 960
height = 540
window_width = window.winfo_screenwidth()
window_height = window.winfo_screenheight()
left = int((window_width - width)/2)
top = int((window_height - height)/2) 
Str_Len = 30

window.title('MIPS instruction decoder 2') 
window.geometry(f'{width}x{height}+{left}+{top}') # window show size
window.configure(bg='#fff') # window BG color
label = tk.Label(window, bd=10, bg='#ffa',fg='#000', text = "MIPS instruction decoder").pack(side="top",fill="x") # Title Label 

# input block
Input_label = tk.Label(window, bg='#fff', text='Please enter MIPS instruction:',font=("Times",14,"bold"))
Input_label.place(relx=0.2,y=100)

entry = tk.Entry(window,width=Str_Len,font=("Times",16)) 
entry.place(relx=0.6,y=100)

button = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(entry),text='Decode', font=("Times",10)) # button  
button.place(relx=0.5,y=140,anchor = 'center')


def show(self):
    global User_input,respond
    User_input = self.get()
    User_input = ''.join(User_input.replace(' ',',',1))
    User_input = User_input.strip( ')' )
    executeInstr(parse(User_input))
    
    if type(respond) != str:
        text = tk.Text(window,height=10, font=("Times",14,"bold"))
        for res in respond:
            text.insert(tk.INSERT,f"{res[0]}: {res[1]}\n")
        text.place(relx=0.2,rely=0.5)
#         Out_label = tk.Label(window , text=Txt, font=("Times",14,"bold"), bd=10 , fg='#225', bg= "#fff",justify = 'left',anchor = 'w')
#         Out_label.place(relx=0.2,rely=0.5)

    else:
         print(type(respond))
#         Out_label.configure(text=f"{ans}",bd=152 , fg='#225', bg= "#aaa",anchor = 'center')
    return respond
  
window.mainloop() 
print(User_input)